# Scrape Upcoming Events

This script is dedicated to scraping and collecting upcoming Ultimate Fighting Championship (UFC) event and fighter data from the official UFC website (ufc.com) and UFC statistics site (ufcstats.com). Here is a summary of what each part of the code does:

1. The script starts with importing essential libraries like pandas, numpy, selenium, BeautifulSoup, etc.

2. The `get_next_events` function is designed to scrape upcoming events' data from the UFC website. It collects the title, URL, and date for each event.

3. The `get_event_fights` function gathers information about each fight happening in an event. It extracts details such as fighters' names, weight class, and odds.

4. The `secret_number` function uses Selenium and BeautifulSoup to automate browser actions to navigate the website and find a particular number, which seems to be specific to each event's matchup.

5. The `get_next_events2` function is similar to `get_next_events`, but it also removes events that lack fight data.

6. The `get_next_event_ufcstats` function scrapes data about upcoming events from the UFC Stats website.

7. The `get_fighter_urls` function collects fighter-related data such as name, URL of the profile from the UFC Stats website.

8. The `find_all_iframe_sources` function finds the source URLs for all iframes on a given UFC event page.

9. The `get_iframe_src_data_2` function uses the previously found iframe source URLs to gather detailed statistics about each fighter.

10. Each function gets applied iteratively to events and matchups, storing its collected information in pandas DataFrames for convenient manipulation and analysis.

11. Finally, the program saves the collated and cleaned data to a CSV file for further use.


In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from random import randint
import  random
import os
os.chdir('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from cmath import nan
import urllib
import urllib.request
import re
import time


In [2]:
def get_next_events(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('div', class_='c-card-event--result__info')
    event1_txt = soup.find('div', class_='c-card-event--result__info').text
    event1_url = event1.find('a')['href']
    event1_url = 'https://www.ufc.com' + event1_url
    event1_title = event1_txt.split('\n')[1]
    event1_time = event1_txt.split('/')[1]

    data = pd.DataFrame({'event_title': [event1_title], 'event_url': [event1_url], 'event_date': [event1_time]})

    event2 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event2_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event2_url = event2.find('a')['href']
    event2_url = 'https://www.ufc.com' + event2_url
    event2_title = event2_txt.split('\n')[1]
    event2_time = event2_txt.split('/')[1]


    data = data.append({'event_title': event2_title, 'event_url': event2_url, 'event_date': event2_time}, ignore_index=True)
    
    event3 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event3_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event3_url = event3.find('a')['href']
    event3_url = 'https://www.ufc.com' + event3_url
    event3_title = event3_txt.split('\n')[1]
    event3_time = event3_txt.split('/')[1]

    data = data.append({'event_title': event3_title, 'event_url': event3_url, 'event_date': event3_time}, ignore_index=True)
    
    return data

# Function to get the fight card for a given event using BeautifulSoup
def get_event_fights(event_url):
    page = requests.get(event_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get main card, fight 1

    mcn = soup.find_all('li', class_='l-listing__item')
    # get num of mc
    num_mc = len(mcn)
    # for each mc, do the following
    data = pd.DataFrame()
    n = 0
    for i in mcn:
        mc = mcn[n]
        # fight 1
        fighter1= mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--red').text
        fighter1 = fighter1.replace('\n', ' ')
        fighter1 = fighter1.strip()
        fighter2 = mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--blue').text
        fighter2 = fighter2.replace('\n', ' ')
        fighter2 = fighter2.strip()
        weightclass = mc.find('div', class_='c-listing-fight__class-text').text
        fighter1_odds = mc.find('span', class_='c-listing-fight__odds').text
        fighter2_odds = mc.find('span', class_='c-listing-fight__odds').find_next('span', class_='c-listing-fight__odds').text
        fighter1_odds = fighter1_odds.replace('\n', '')
        fighter2_odds = fighter2_odds.replace('\n', '')
        # fighter odds to float
        if (fighter1_odds == '-') :
            fighter1_odds = nan
        if (fighter2_odds == '-') :
            fighter2_odds = nan

        data = data.append({'fighter1': fighter1, 'fighter2': fighter2, 'weightclass': weightclass, 
                            'fighter1_odds': fighter1_odds, 'fighter2_odds': fighter2_odds}, ignore_index=True)
        n = n + 1
    return data

# Find secret number in ufc events using BS & Selenium
def secret_number(event_url):
    # if no driver open, open one
    driver = None
    if (driver == None):
        driver = webdriver.Chrome('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\UFC_Prediction_V2\\chromedriver.exe')
    else:
        driver = driver
    
    driver.get(event_url)
    time.sleep(3)
    # click the first matchup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pretty = soup.prettify()
    # find first data-fmid to get first matchup
    fmid_start = pretty.find('data-fmid')
    fmid = pretty[fmid_start+11:fmid_start+16]
    driver.get(event_url +'#' + fmid)
    time.sleep(6)
    # find all links within page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find all iframe src
    iframe = soup.find_all('iframe')
    # find all links
    iframe_text = str(iframe)
    matchup = iframe_text.find('matchup')
    matchup_url = iframe_text[matchup+8:matchup+12]
    print('matchup_url: ' + matchup_url)
    secret_number = matchup_url
    return matchup

# get next events if event fighter data is not na
def get_next_events2(url):
    data = get_next_events(url)
    for i in range(0, len(data)):
        event_url = data['event_url'][i]
        event_fights = get_event_fights(event_url)
        if (len(event_fights) == 0):
            data = data.drop(i)
    return data

# get next events from UFCStats.com using BS
def get_next_event_ufcstats():
    url = 'http://www.ufcstats.com/statistics/events/upcoming'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('td', class_='b-statistics__table-col')
    event1_txt = soup.find('td', class_='b-statistics__table-col').text
    event_txt = event1_txt.replace('   ', '').replace('\n', '').strip()
    event_title = event_txt.split('  ')[0]
    event_date = event_txt.split('  ')[1]
    event1_url = event1.find('a')['href']
    data = pd.DataFrame({'event_title': [event_title], 'event_url': [event1_url], 'event_date': [event_date]})
    return data


# get fighter urls from UFCStats.com using BS
def get_fighter_urls(event_details_url):
    page = requests.get(event_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    next_event_data = pd.DataFrame()

    for event in events:
        fighters = events[n].find_all('p', class_='b-fight-details__table-text')
        fighter1 = fighters[0].text
        fighter1 = fighter1.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text
        fighter2 = fighter2.replace('  ', '').replace('\n', '').strip()
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']
        next_event_data = next_event_data.append({'fighter1' :fighter1, 'fighter2:' : fighter2, 'fighter1_url': fighter1_url, 'fighter2_url':fighter2_url, 'fight#' : n+1}, ignore_index = True)
        n += 1

    return next_event_data



In [3]:
next_events = get_next_events2('https://www.ufc.com/events')
next_events

,event_title,event_url,event_date
0,Gane vs Spivac,https://www.ufc.com/event/ufc-fight-night-sept...,3:00 PM EDT
1,Adesanya vs Strickland,https://www.ufc.com/event/ufc-293,10:00 PM EDT
2,Grasso vs Shevchenko 2,https://www.ufc.com/event/ufc-fight-night-sept...,10:00 PM EDT


In [4]:
event_url =  next_events['event_url'][0]

In [5]:
next_event_title = next_events['event_title'][0]

In [6]:
next_event = get_event_fights(event_url)

In [7]:
page = requests.get(event_url)
soup = BeautifulSoup(page.content, 'html.parser')
h = soup.find_all('div', class_='c-listing-fight')

data_fmid = []
for i in h:
    data_fmid.append(i['data-fmid'])

next_event['fight_number'] = data_fmid[:len(next_event)]
next_event['matchup_url'] = event_url +'#' + next_event['fight_number'].astype(str)

next_event

,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url
0,Ciryl Gane,Serghei Spivac,Heavyweight Bout,-180,+150,10700,https://www.ufc.com/event/ufc-fight-night-sept...
1,Manon Fiorot,Rose Namajunas,Women's Flyweight Bout,-198,+164,10731,https://www.ufc.com/event/ufc-fight-night-sept...
2,Benoît Saint-Denis,Thiago Moises,Lightweight Bout,-135,+114,10794,https://www.ufc.com/event/ufc-fight-night-sept...
3,Volkan Oezdemir,Bogdan Guskov,Light Heavyweight Bout,NaN,NaN,10858,https://www.ufc.com/event/ufc-fight-night-sept...
4,William Gomis,Lucas Almeida,Featherweight Bout,-166,+140,10760,https://www.ufc.com/event/ufc-fight-night-sept...
5,Yanis Ghemmouri,Caolán Loughran,Bantamweight Bout,NaN,NaN,10793,https://www.ufc.com/event/ufc-fight-night-sept...
6,Morgan Charriere,Manolo Zecchini,Featherweight Bout,NaN,NaN,10826,https://www.ufc.com/event/ufc-fight-night-sept...
7,Taylor Lapilus,Muin Gafurov,Bantamweight Bout,-192,+160,10859,https://www.ufc.com/event/ufc-fight-night-sept...
8,Zarah Fairn,Jacqueline Cavalcanti,Women's Bantamweight Bout,NaN,NaN,10860,https://www.ufc.com/event/ufc-fight-night-sept...
9,Nora Cornolle,Joselyne Edwards,Women's Bantamweight Bout,NaN,NaN,10721,https://www.ufc.com/event/ufc-fight-night-sept...


In [8]:
def find_all_iframe_sources(matchup_url):
    driver = webdriver.Chrome('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/V2_Newer_Notebooks/chromedriver')
    driver.get(matchup_url)
    time.sleep(3)
    # get innerhtml
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pretty = soup.prettify()
    # find all links within page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find all iframe src
    iframe = soup.find_all('iframe')
    # list all links
    iframe_text = str(iframe)
    # separate links
    iframe_text = iframe_text.split('src="')
    iframe_text = iframe_text[1:]
    iframe_text = [i.split('"')[0] for i in iframe_text]
    # only keep links that contain matchup
    iframe_text = [i for i in iframe_text if 'matchup' in i]
    # only keep top link
    iframe_text = iframe_text[0]
    


    return iframe_text


In [9]:
find_all_iframe_sources(next_event['matchup_url'][1])

'/matchup/1159/10731/pre'

Add iframe Sources to next_event. This can take a few minutes. 

In [10]:
# add iframe sources to df  with itterrows and apply
next_event['iframe_src'] = next_event.apply(lambda x: find_all_iframe_sources(x['matchup_url']), axis=1)

In [ ]:
next_event

,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url,iframe_src
0,Max Holloway,Chan Sung Jung,Featherweight Bout,-850,+520,10699,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10699/pre
1,Anthony Smith,Ryan Spann,Light Heavyweight Bout,NaN,NaN,10729,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10729/pre
2,Giga Chikadze,Alex Caceres,Featherweight Bout,NaN,NaN,10830,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10830/pre
3,Rinya Nakamura,Fernie Garcia,Bantamweight Bout,NaN,NaN,10730,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10730/pre
4,Erin Blanchfield,Taila Santos,Women's Flyweight Bout,NaN,NaN,10754,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10754/pre
5,Junior Tafa,Parker Porter,Heavyweight Bout,NaN,NaN,10757,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10757/pre
6,Waldo Cortes Acosta,Lukasz Brzeski,Heavyweight Bout,NaN,NaN,10758,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10758/pre
7,Toshiomi Kazama,Garrett Armfield,Bantamweight Bout,NaN,NaN,10831,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10831/pre
8,Chidi Njokuani,Michal Oleksiejczuk,Middleweight Bout,NaN,NaN,10735,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10735/pre
9,Song Kenan,Rolando Bedoya,Welterweight Bout,NaN,NaN,10756,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10756/pre


In [ ]:
next_event['full_iframe_src'] = 'https://www.ufc.com' + next_event['iframe_src']

In [ ]:
def get_iframe_src_data_2(iframe_src):
    iframe_response = requests.get(iframe_src)
    iframe_soup = BeautifulSoup(iframe_response.content, 'html.parser')
    # get all c-stat-compare__group-1 red
    red = iframe_soup.find_all('div', class_='c-stat-compare__group-1 red')
    # return all red texts
    red_text = [i.text for i in red]
    # assign to variables
    red_record = red_text[0]
    red_last_fight = red_text[1]
    red_country = red_text[2]
    red_height = red_text[3]
    red_weight = red_text[4]
    red_reach = red_text[5]
    red_legreach = red_text[6]
    red_win_by_ko_percent = red_text[7]
    red_win_by_sub_percent = red_text[8]
    red_win_by_dec_percent = red_text[9]
    red_avg_fight_time = red_text[10]
    red_knockdowns_per_15_min = red_text[11]
    # sig strikes
    red_sig_strikes_landed_per_min = red_text[12]
    red_sig_strikes_percent = red_text[13]
    red_sig_strikes_absorbed_per_min = red_text[14]
    red_sig_strikes_absorbed_percent = red_text[15]
    # grappling
    red_takedowns_landed_per_15_min = red_text[16]
    red_takedown_accuracy = red_text[17]
    red_takedown_defense = red_text[18]
    red_submissions_attempts_per_15_min = red_text[19]
    # odds
    red_moneyline = red_text[20]


    # make df for red
    red_df = pd.DataFrame({'red_record': red_record, 'red_last_fight': red_last_fight, 
                            'red_country': red_country, 'red_height': red_height, 'red_weight': red_weight, 
                            'red_reach': red_reach, 'red_legreach': red_legreach, 
                            'red_win_by_ko_percent': red_win_by_ko_percent, 
                            'red_win_by_sub_percent': red_win_by_sub_percent, 'red_win_by_dec_percent': red_win_by_dec_percent, 
                            'red_avg_fight_time': red_avg_fight_time, 'red_knockdowns_per_15_min': red_knockdowns_per_15_min, 
                            'red_sig_strikes_landed_per_min': red_sig_strikes_landed_per_min, 
                            'red_sig_strikes_percent': red_sig_strikes_percent, 
                            'red_sig_strikes_absorbed_per_min': red_sig_strikes_absorbed_per_min, 
                            'red_sig_strikes_absorbed_percent': red_sig_strikes_absorbed_percent, 
                            'red_takedowns_landed_per_15_min': red_takedowns_landed_per_15_min, 
                            'red_takedown_accuracy': red_takedown_accuracy, 'red_takedown_defense': red_takedown_defense, 
                            'red_submissions_attempts_per_15_min': red_submissions_attempts_per_15_min, 
                            'red_moneyline': red_moneyline}, index=[0])

    # clean all values in red_df, removing all \n 
    red_df = red_df.applymap(lambda x: x.replace('\n', ''))
    # strip all values in red_df
    red_df = red_df.applymap(lambda x: x.strip())

    # get all c-stat-compare__group-1 blue
    blue = iframe_soup.find_all('div', class_='c-stat-compare__group-2 blue')
    # return all blue texts
    blue_text = [i.text for i in blue]
    # assign to variables
    blue_record = blue_text[0]
    blue_last_fight = blue_text[1]
    blue_country = blue_text[2]
    blue_height = blue_text[3]
    blue_weight = blue_text[4]
    blue_reach = blue_text[5]
    blue_legreach = blue_text[6]
    blue_win_by_ko_percent = blue_text[7]
    blue_win_by_sub_percent = blue_text[8]
    blue_win_by_dec_percent = blue_text[9]
    blue_avg_fight_time = blue_text[10]
    blue_knockdowns_per_15_min = blue_text[11]
    # sig strikes
    blue_sig_strikes_landed_per_min = blue_text[12]
    blue_sig_strikes_percent = blue_text[13]
    blue_sig_strikes_absorbed_per_min = blue_text[14]
    blue_sig_strikes_absorbed_percent = blue_text[15]
    # grappling
    blue_takedowns_landed_per_15_min = blue_text[16]
    blue_takedown_accuracy = blue_text[17]
    blue_takedown_defense = blue_text[18]
    blue_submissions_attempts_per_15_min = blue_text[19]
    # odds
    blue_moneyline = blue_text[20]


    # make df for blue
    blue_df = pd.DataFrame({'blue_record': blue_record, 'blue_last_fight': blue_last_fight,
                            'blue_country': blue_country, 'blue_height': blue_height, 'blue_weight': blue_weight,
                            'blue_reach': blue_reach, 'blue_legreach': blue_legreach,
                            'blue_win_by_ko_percent': blue_win_by_ko_percent,
                            'blue_win_by_sub_percent': blue_win_by_sub_percent, 'blue_win_by_dec_percent': blue_win_by_dec_percent,
                            'blue_avg_fight_time': blue_avg_fight_time, 'blue_knockdowns_per_15_min': blue_knockdowns_per_15_min,
                            'blue_sig_strikes_landed_per_min': blue_sig_strikes_landed_per_min,
                            'blue_sig_strikes_percent': blue_sig_strikes_percent,
                            'blue_sig_strikes_absorbed_per_min': blue_sig_strikes_absorbed_per_min,
                            'blue_sig_strikes_absorbed_percent': blue_sig_strikes_absorbed_percent,
                            'blue_takedowns_landed_per_15_min': blue_takedowns_landed_per_15_min,
                            'blue_takedown_accuracy': blue_takedown_accuracy, 'blue_takedown_defense': blue_takedown_defense,
                            'blue_submissions_attempts_per_15_min': blue_submissions_attempts_per_15_min,
                            'blue_moneyline': blue_moneyline}, index=[0])

    # clean all values in blue_df, removing all \n
    blue_df = blue_df.applymap(lambda x: x.replace('\n', ''))
    # strip all values in blue_df
    blue_df = blue_df.applymap(lambda x: x.strip())

    # # transpose both dfs, then merge]
    # red_df = red_df.T
    # # rename column 'red'
    # red_df.columns = ['red']
    # # rename rows, removing 'red_'
    # red_df.index = red_df.index.str.replace('red_', '')
    # # transpose blue_df
    # blue_df = blue_df.T
    # # rename column 'blue'
    # blue_df.columns = ['blue']
    # # rename rows, removing 'blue_'
    # blue_df.index = blue_df.index.str.replace('blue_', '')

    # # merge red_df and blue_df
    # df = pd.concat([red_df, blue_df], axis=1)

    # append blue_df to red_df by axis=1
    dfs = pd.concat([red_df, blue_df], axis=1)

    # return df
    return dfs




In [ ]:
# get iframe src data for all events

event_data = []

for i in range(len(next_event['full_iframe_src'])):
    event_data.append(get_iframe_src_data_2(next_event['full_iframe_src'][i]))

# make df from event_data
event_data_df = pd.concat(event_data, axis=0)

event_data_df

,red_record,red_last_fight,red_country,red_height,red_weight,red_reach,red_legreach,red_win_by_ko_percent,red_win_by_sub_percent,red_win_by_dec_percent,...,blue_knockdowns_per_15_min,blue_sig_strikes_landed_per_min,blue_sig_strikes_percent,blue_sig_strikes_absorbed_per_min,blue_sig_strikes_absorbed_percent,blue_takedowns_landed_per_15_min,blue_takedown_accuracy,blue_takedown_defense,blue_submissions_attempts_per_15_min,blue_moneyline
0,24-7-0,Win,United States,"5' 11""",146.00 LB,69 in,42 in,43%,9%,48%,...,0.46,3.97,42%,4.35,52%,0.74,47%,72%,0.65,+520
0,36-18-0,Loss,United States,"6' 4""",205.50 LB,76 in,44 in,53%,39%,8%,...,1.64,3.28,44%,3.32,48%,1.64,37%,45%,1.91,N/A
0,14-3-0,Loss,Georgia,"6' 0""",146.00 LB,74 in,40.5 in,64%,7%,29%,...,0.23,4.16,51%,2.87,63%,0.59,68%,64%,0.73,N/A
0,7-0-0,Win,Japan,"5' 7""",135.00 LB,68.5 in,37 in,71%,14%,14%,...,0.47,2.18,39%,2.83,54%,0.47,50%,61%,0.47,N/A
0,11-1-0,Win,United States,"5' 4""",125.50 LB,66 in,36 in,18%,36%,45%,...,0.43,3.35,50%,2.43,59%,2.29,88%,78%,1.29,N/A
0,4-1-0,Loss,Australia,"6' 3""",252.00 LB,72 in,0 in,100%,0%,0%,...,0,6.66,50%,6.34,51%,1.38,27%,66%,0.46,N/A
0,9-1-0,Loss,Dominican Republic,"6' 4""",258.00 LB,70 in,43.5 in,50%,13%,38%,...,0,5.11,47%,3.35,56%,0.68,50%,42%,0.68,N/A
0,10-3-0,Loss,Japan,"5' 7""",136.00 LB,69 in,38.5 in,30%,50%,20%,...,0,2.56,29%,3.53,59%,0,0%,66%,0,N/A
0,"22-9-0, 1NC",Loss,United States,"6' 3""",185.00 LB,80 in,45 in,64%,5%,32%,...,1.4,4.82,49%,4.02,62%,0.6,37%,44%,0,N/A
0,19-7-0,China,"6' 0""",171.00 LB,71.5 in,41 in,47%,42%,11%,8:26,...,9.93,71%,8.67,54%,0,0%,0%,0,N/A,N/A


In [ ]:
# reindex both
next_event = next_event.reset_index(drop=True)
event_data_df = event_data_df.reset_index(drop=True)

# Append event_data_df to next_event
next_event = pd.concat([next_event, event_data_df], axis=1)
next_event

,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url,iframe_src,full_iframe_src,red_record,...,blue_knockdowns_per_15_min,blue_sig_strikes_landed_per_min,blue_sig_strikes_percent,blue_sig_strikes_absorbed_per_min,blue_sig_strikes_absorbed_percent,blue_takedowns_landed_per_15_min,blue_takedown_accuracy,blue_takedown_defense,blue_submissions_attempts_per_15_min,blue_moneyline
0,Max Holloway,Chan Sung Jung,Featherweight Bout,-850,+520,10699,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10699/pre,https://www.ufc.com/matchup/1161/10699/pre,24-7-0,...,0.46,3.97,42%,4.35,52%,0.74,47%,72%,0.65,+520
1,Anthony Smith,Ryan Spann,Light Heavyweight Bout,NaN,NaN,10729,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10729/pre,https://www.ufc.com/matchup/1161/10729/pre,36-18-0,...,1.64,3.28,44%,3.32,48%,1.64,37%,45%,1.91,N/A
2,Giga Chikadze,Alex Caceres,Featherweight Bout,NaN,NaN,10830,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10830/pre,https://www.ufc.com/matchup/1161/10830/pre,14-3-0,...,0.23,4.16,51%,2.87,63%,0.59,68%,64%,0.73,N/A
3,Rinya Nakamura,Fernie Garcia,Bantamweight Bout,NaN,NaN,10730,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10730/pre,https://www.ufc.com/matchup/1161/10730/pre,7-0-0,...,0.47,2.18,39%,2.83,54%,0.47,50%,61%,0.47,N/A
4,Erin Blanchfield,Taila Santos,Women's Flyweight Bout,NaN,NaN,10754,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10754/pre,https://www.ufc.com/matchup/1161/10754/pre,11-1-0,...,0.43,3.35,50%,2.43,59%,2.29,88%,78%,1.29,N/A
5,Junior Tafa,Parker Porter,Heavyweight Bout,NaN,NaN,10757,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10757/pre,https://www.ufc.com/matchup/1161/10757/pre,4-1-0,...,0,6.66,50%,6.34,51%,1.38,27%,66%,0.46,N/A
6,Waldo Cortes Acosta,Lukasz Brzeski,Heavyweight Bout,NaN,NaN,10758,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10758/pre,https://www.ufc.com/matchup/1161/10758/pre,9-1-0,...,0,5.11,47%,3.35,56%,0.68,50%,42%,0.68,N/A
7,Toshiomi Kazama,Garrett Armfield,Bantamweight Bout,NaN,NaN,10831,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10831/pre,https://www.ufc.com/matchup/1161/10831/pre,10-3-0,...,0,2.56,29%,3.53,59%,0,0%,66%,0,N/A
8,Chidi Njokuani,Michal Oleksiejczuk,Middleweight Bout,NaN,NaN,10735,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10735/pre,https://www.ufc.com/matchup/1161/10735/pre,"22-9-0, 1NC",...,1.4,4.82,49%,4.02,62%,0.6,37%,44%,0,N/A
9,Song Kenan,Rolando Bedoya,Welterweight Bout,NaN,NaN,10756,https://www.ufc.com/event/ufc-fight-night-augu...,/matchup/1161/10756/pre,https://www.ufc.com/matchup/1161/10756/pre,19-7-0,...,9.93,71%,8.67,54%,0,0%,0%,0,N/A,N/A


In [ ]:
# save to csv
next_event.to_csv('data/final/next_fights/'+ next_event_title +'_.csv', index=False)